In [1]:
import pandas as pd
from scipy import sparse

In [2]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.merge(movies,ratings).drop(['genres','timestamp'],axis=1)
print(ratings.shape)
ratings.head()

(100836, 4)


movieId             title  userId  rating
0        1  Toy Story (1995)       1     4.0
1        1  Toy Story (1995)       5     4.0
2        1  Toy Story (1995)       7     4.5
3        1  Toy Story (1995)      15     2.5
4        1  Toy Story (1995)      17     4.5

In [3]:
userRatings = ratings.pivot_table(index=['userId'],columns=['title'],values='rating')
userRatings.head()
print("Before: ",userRatings.shape)
userRatings = userRatings.dropna(thresh=10, axis=1).fillna(0,axis=1)
print("After: ",userRatings.shape)

Before:  (610, 9719)
After:  (610, 2269)


In [6]:
corrMatrix = userRatings.corr(method='pearson')
corrMatrix.head(5)

title                              'burbs, The (1989)  \
title                                                   
'burbs, The (1989)                           1.000000   
(500) Days of Summer (2009)                  0.063117   
10 Cloverfield Lane (2016)                  -0.023768   
10 Things I Hate About You (1999)            0.143482   
10,000 BC (2008)                             0.011998   

title                              (500) Days of Summer (2009)  \
title                                                            
'burbs, The (1989)                                    0.063117   
(500) Days of Summer (2009)                           1.000000   
10 Cloverfield Lane (2016)                            0.142471   
10 Things I Hate About You (1999)                     0.273989   
10,000 BC (2008)                                      0.193960   

title                              10 Cloverfield Lane (2016)  \
title                                                           
'burbs, The (1989)                                  -0.023768   
(500) Days of Summer (2009)                          0.142471   
10 Cloverfield Lane (2016)                           1.000000   
10 Things I Hate About You (1999)                   -0.005799   
10,000 BC (2008)                                     0.112396   

title                              10 Things I Hate About You (1999)  \
title                                                                  
'burbs, The (1989)                                          0.143482   
(500) Days of Summer (2009)                                 0.273989   
10 Cloverfield Lane (2016)                                 -0.005799   
10 Things I Hate About You (1999)                           1.000000   
10,000 BC (2008)                                            0.244670   

title                              10,000 BC (2008)  101 Dalmatians (1996)  \
title                                                                        
'burbs, The (1989)                         0.011998               0.087931   
(500) Days of Summer (2009)                0.193960               0.148903   
10 Cloverfield Lane (2016)                 0.112396               0.006139   
10 Things I Hate About You (1999)          0.244670               0.223481   
10,000 BC (2008)                           1.000000               0.234459   

title                              101 Dalmatians (One Hundred and One Dalmatians) (1961)  \
title                                                                                       
'burbs, The (1989)                                                          0.224052        
(500) Days of Summer (2009)                                                 0.142141        
10 Cloverfield Lane (2016)                                                 -0.016835        
10 Things I Hate About You (1999)                                           0.211473        
10,000 BC (2008)                                                            0.119132        

title                              12 Angry Men (1957)  \
title                                                    
'burbs, The (1989)                            0.034223   
(500) Days of Summer (2009)                   0.159756   
10 Cloverfield Lane (2016)                    0.031704   
10 Things I Hate About You (1999)             0.011784   
10,000 BC (2008)                              0.059187   

title                              12 Years a Slave (2013)  127 Hours (2010)  \
title                                                                          
'burbs, The (1989)                                0.009277          0.008331   
(500) Days of Summer (2009)                       0.135486          0.200135   
10 Cloverfield Lane (2016)                       -0.024275          0.272943   
10 Things I Hate About You (1999)                 0.091964          0.043383   
10,000 BC (2008)                                 -0.025882          0.089328   

title    

In [7]:
def get_similar(movie_name,rating):
    similar_ratings = corrMatrix[movie_name]*(rating-2.5)
    similar_ratings = similar_ratings.sort_values(ascending=False)
    return similar_ratings

In [8]:
romantic_lover = [("(500) Days of Summer (2009)",5),("Alice in Wonderland (2010)",3),("Aliens (1986)",1),("2001: A Space Odyssey (1968)",2)]
similar_movies = pd.DataFrame()
for movie,rating in romantic_lover:
    similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)

similar_movies.head(10)


'burbs, The (1989)  (500) Days of Summer (2009)  \
0            0.157792                     2.500000   
1           -0.016276                     0.203998   
2           -0.304722                    -0.062634   
3           -0.102988                    -0.056808   

   10 Cloverfield Lane (2016)  10 Things I Hate About You (1999)  \
0                    0.356179                           0.684973   
1                    0.126834                           0.113241   
2                   -0.214700                          -0.118754   
3                   -0.049655                          -0.042987   

   10,000 BC (2008)  101 Dalmatians (1996)  \
0          0.484900               0.372257   
1          0.092218               0.085790   
2         -0.037059              -0.063992   
3         -0.021729              -0.055422   

   101 Dalmatians (One Hundred and One Dalmatians) (1961)  \
0                                           0.355353        
1                                           0.072825        
2                                          -0.170195        
3                                          -0.051115        

   12 Angry Men (1957)  12 Years a Slave (2013)  127 Hours (2010)  ...  \
0             0.399389                 0.338715          0.500338  ...   
1             0.097794                 0.083822          0.084897  ...   
2            -0.280090                -0.016283         -0.102493  ...   
3            -0.097954                -0.061595         -0.070398  ...   

   Zack and Miri Make a Porno (2008)  Zero Dark Thirty (2012)  \
0                           0.936288                 0.446637   
1                           0.159907                 0.085502   
2                          -0.147339                -0.162387   
3                          -0.075325                -0.048607   

   Zero Effect (1998)  Zodiac (2007)  Zombieland (2009)  Zoolander (2001)  \
0            0.171018       1.036463           0.889309          0.630565   
1            0.011564       0.176888           0.168302          0.121710   
2           -0.368712      -0.281119          -0.263686         -0.228562   
3           -0.128795      -0.175166          -0.098088         -0.074205   

   Zootopia (2016)  eXistenZ (1999)  xXx (2002)  ¡Three Amigos! (1986)  
0         0.540017         0.134034    0.602730               0.314763  
1         0.072590         0.025695    0.081764               0.027540  
2        -0.144292        -0.410899   -0.242935              -0.238410  
3        -0.049626        -0.153017   -0.082048              -0.091432  

[4 rows x 2269 columns]

In [9]:
similar_movies.sum().sort_values(ascending=False).head(20)

(500) Days of Summer (2009)                      2.584556
Alice in Wonderland (2010)                       1.395229
Silver Linings Playbook (2012)                   1.254800
Yes Man (2008)                                   1.116264
Adventureland (2009)                             1.112235
Marley & Me (2008)                               1.108381
About Time (2013)                                1.102192
Crazy, Stupid, Love. (2011)                      1.088757
50/50 (2011)                                     1.086517
Help, The (2011)                                 1.075963
Up in the Air (2009)                             1.053037
Holiday, The (2006)                              1.034470
Friends with Benefits (2011)                     1.030875
Notebook, The (2004)                             1.025880
Easy A (2010)                                    1.015771
Secret Life of Walter Mitty, The (2013)          0.997979
Perks of Being a Wallflower, The (2012)          0.967425
Toy Story 3 (2

In [10]:
action_lover = [("Amazing Spider-Man, The (2012)",5),("Mission: Impossible III (2006)",4),("Toy Story 3 (2010)",2),("2 Fast 2 Furious (Fast and the Furious 2, The) (2003)",4)]
similar_movies = pd.DataFrame()
for movie,rating in action_lover:
    similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)

similar_movies.head(10)
similar_movies.sum().sort_values(ascending=False).head(5)

Amazing Spider-Man, The (2012)                           3.233134
Mission: Impossible III (2006)                           2.874798
2 Fast 2 Furious (Fast and the Furious 2, The) (2003)    2.701477
Over the Hedge (2006)                                    2.229721
Crank (2006)                                             2.176259
dtype: float64